In [1]:
%run ../wikiqa/init.ipynb

matchzoo version 2.2.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

import matchzoo as mz
from matchzoo.contrib.models.esim import ESIM

In [3]:
def load_filtered_data(preprocessor, data_type):
    assert ( data_type in ['train', 'dev', 'test'])
    data_pack = mz.datasets.wiki_qa.load_data(data_type, task='ranking')

    if data_type == 'train':
        X, Y = preprocessor.fit_transform(data_pack).unpack()
    else:
        X, Y = preprocessor.transform(data_pack).unpack()

    new_idx = []
    for i in range(Y.shape[0]):
        if X["length_left"][i] == 0 or X["length_right"][i] == 0:
            continue
        new_idx.append(i)
    new_idx = np.array(new_idx)
    print("Removed empty data. Found ", (Y.shape[0] - new_idx.shape[0]))

    for k in X.keys():
        X[k] = X[k][new_idx]
    Y = Y[new_idx]

    pos_idx = (Y == 1)[:, 0]
    pos_qid = X["id_left"][pos_idx]
    keep_idx_bool = np.array([ qid in pos_qid for qid in X["id_left"]])
    keep_idx = np.arange(keep_idx_bool.shape[0])
    keep_idx = keep_idx[keep_idx_bool]
    print("Removed questions with no pos label. Found ", (keep_idx_bool == 0).sum())

    print("shuffling...")
    np.random.shuffle(keep_idx)
    for k in X.keys():
        X[k] = X[k][keep_idx]
    Y = Y[keep_idx]

    return X, Y, preprocessor

In [4]:
fixed_length_left = 10
fixed_length_right = 40
batch_size = 32
epochs = 5

In [5]:
# prepare data
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=fixed_length_left,
                                                  fixed_length_right=fixed_length_right,
                                                  remove_stop_words=False,
                                                  filter_low_freq=10)

train_X, train_Y, preprocessor = load_filtered_data(preprocessor, 'train')
val_X, val_Y, _ = load_filtered_data(preprocessor, 'dev')
pred_X, pred_Y = val_X, val_Y
# pred_X, pred_Y, _ = load_filtered_data(preprocessor, 'test') # no prediction label for quora dataset

embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'], initializer=lambda: 0)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 15326.69it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:02<00:00, 8165.72it/s]
Building Vocabulary from a datapack.: 100%|██████████| 425518/425518 [00:00<00:00, 5865915.93it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 14423.46it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:02<00:00, 8559.19it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 156746.57it/s]
Removed empty data. Found  59
Removed questions with no pos label. Found  11653
shuffling...
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 296/296 [00:00<00:00, 15702.84it/s]
Processing text_righ

In [6]:
model = ESIM()
model.params['task'] = mz.tasks.Ranking()
model.params['mask_value'] = 0
model.params['input_shapes'] = [[fixed_length_left, ],
                                [fixed_length_right, ]]
model.params['lstm_dim'] = 300
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = False
model.params['dropout_rate'] = 0.5

model.params['mlp_num_units'] = 300
model.params['mlp_num_layers'] = 0
model.params['mlp_num_fan_out'] = 300
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = Adam(lr=4e-4)

model.guess_and_fill_missing_params()
model.build()

model.compile()
model.backend.summary() # not visualize

/home/songzy/.virtualenvs/matchzoo/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          [(None, 10)]         0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         [(None, 40)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1959300     text_left[0][0]                  
                                                                 text_right[0][

In [7]:
# run as classification task
model.load_embedding_matrix(embedding_matrix)
evaluate = mz.callbacks.EvaluateAllMetrics(model,
                                           x=pred_X,
                                           y=pred_Y,
                                           once_every=1,
                                           batch_size=len(pred_Y))

history = model.fit(x = [train_X['text_left'],
                         train_X['text_right']],
                    y = train_Y,
                    validation_data = (val_X, val_Y),
                    batch_size = batch_size,
                    epochs = epochs,
                    callbacks=[evaluate]
                    )

Epoch 1/5
271/271 [==============================] - ETA: 0s - loss: 0.1059/home/songzy/.virtualenvs/matchzoo/lib/python3.8/site-packages/tensorflow/python/keras/engine/functional.py:588: UserWarning: Input dict contained keys ['id_left', 'length_left', 'id_right', 'length_right'] which did not match any model input. They will be ignored by the model.
  warnings.warn(
271/271 [==============================] - 68s 229ms/step - loss: 0.1059 - val_loss: 0.0952
Validation: mean_average_precision(0.0): 0.6286666265832934
Epoch 2/5
271/271 [==============================] - 64s 235ms/step - loss: 0.0970 - val_loss: 0.0915
Validation: mean_average_precision(0.0): 0.6482657107695167
Epoch 3/5
271/271 [==============================] - 66s 245ms/step - loss: 0.0938 - val_loss: 0.0901
Validation: mean_average_precision(0.0): 0.6532234442598588
Epoch 4/5
271/271 [==============================] - 60s 222ms/step - loss: 0.0910 - val_loss: 0.1005
Validation: mean_average_precision(0.0): 0.67003942

In [8]:
# run as classification task
classification_task = mz.tasks.Classification(num_classes=2)
classification_task.metrics = 'acc'

model = ESIM()
model.params['task'] = classification_task
model.params['mask_value'] = 0
model.params['input_shapes'] = [[fixed_length_left, ],
                                [fixed_length_right, ]]
model.params['lstm_dim'] = 300
model.params['embedding_input_dim'] = preprocessor.context['vocab_size']
model.params['embedding_output_dim'] = 300
model.params['embedding_trainable'] = False
model.params['dropout_rate'] = 0.5

model.params['mlp_num_units'] = 300
model.params['mlp_num_layers'] = 0
model.params['mlp_num_fan_out'] = 300
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = Adam(lr=4e-4)

model.guess_and_fill_missing_params()
model.build()

model.compile()
model.backend.summary() # not visualize

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          [(None, 10)]         0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         [(None, 40)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             1959300     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
dropout_1 (Dropout)             multiple             0           embedding[0][0]            

In [9]:
evaluate = mz.callbacks.EvaluateAllMetrics(model,
                                           x=pred_X,
                                           y=pred_Y,
                                           once_every=1,
                                           batch_size=len(pred_Y))

train_Y = to_categorical(train_Y)
val_Y = to_categorical(val_Y)

model.load_embedding_matrix(embedding_matrix)
history = model.fit(x = [train_X['text_left'],
                         train_X['text_right']],
                    y = train_Y,
                    validation_data = (val_X, val_Y),
                    batch_size = batch_size,
                    epochs = epochs,
                    callbacks=[evaluate]
                    )

Epoch 1/5
271/271 [==============================] - 67s 224ms/step - loss: 0.3606 - val_loss: 0.3333
Validation: categorical_accuracy: 0.9884955883026123
Epoch 2/5
271/271 [==============================] - 61s 224ms/step - loss: 0.3266 - val_loss: 0.3345
Validation: categorical_accuracy: 0.9707964658737183
Epoch 3/5
271/271 [==============================] - 60s 223ms/step - loss: 0.3115 - val_loss: 0.3603
Validation: categorical_accuracy: 0.9592920541763306
Epoch 4/5
271/271 [==============================] - 60s 220ms/step - loss: 0.2952 - val_loss: 0.3779
Validation: categorical_accuracy: 0.8716813921928406
Epoch 5/5
271/271 [==============================] - 60s 221ms/step - loss: 0.2833 - val_loss: 0.3188
Validation: categorical_accuracy: 0.9601770043373108


In [10]:
model.evaluate(val_X, val_Y)

{'categorical_accuracy': 0.8929204}